# Original Objective function
$$
E(A_{i}) = \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} k_{ij} q_j \prod_{i' \in \{1, \ldots, m\}}^{\text{msg}} \left( k_{i'j} p_{i'} + 0^{k_{i'j}} \right)
$$

### Simplification assumption
We assume probobility of recieving a tag $q_{j}$ is equal to probobility of recieving message $p_{i}$  
$$q_{j} = p_{i} = p$$

### C1 : Using all tags
Each tag must verify at least one message
$$
\sum_{i=1}^{m} k_{ij} \geq 1 \quad \text{for each } j \in \{1, 2, \ldots, \text{|T|}\}
$$
### C2: Using all messages
Each message must be signed with at least one tag
$$
\sum_{j=1}^{n} k_{ij} \geq 1 \quad \text{for each } i \in \{1, 2, \ldots, \text{|M|}\}
$$

### C3: Define auxiliary varible for number of messages each tag signs
we define number of messages that each tag signs as an auxiliray variable to use later in transforming objective function to a linear model
$$
z_j = \sum_{i=1}^{n} k_{ij} \quad \text{for each } j \in \{1, 2, \ldots, \text{tags}\}
$$
$$
z_j \geq 1 \quad \text{for each } j \in \{1, 2, \ldots, \text{tags}\}
$$


### C4: Re-writing the product part in linear
After simplifying all probobilities to be equal the product part could be linearized using some auxiliary variable and constraints as follow:

$$
\prod_{i' \in \{1, \ldots, m\}}^{\text{msg}} \left( k_{i'j} p_{i'} + 0^{k_{i'j}} \right) = \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} w_{j} p^j
$$

$$
 \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} w_{j} = 1  \quad j \in \{0,1\}

$$
Now the next constraint should be define to force the correcponding $w_j = 1$ which can be done using another auxiliary variable and two constraints as follow
$$z_{j} - j \leq M w_{j}$$
$$j - z_{j} \leq M (1-w_{j})$$


## Simplified Objective function
With linearizing product part, the objective function could be rewritten as: 

$$
E(A_{i}) = \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} k_{ij} p \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} w_{j} p^j 
$$
or even more simple as
$$
E(A_{i}) = \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} k_{ij}\text{ } w_{j}\text{ } p^{(j+1)}

$$

The final part is linearizing the product of the two binary variables $k_{ij}w_{j}$ which could be done this way:

$$x_{ij} \leq k_{ij} $$
$$x_{ij} \leq w_{j} $$
$$x_{ij} \geq k_{ij} + w_{j} -1 $$
$$x_{ij}\in \{0,1\}$$

# Linear Objective function
$$
E(A_{i}) = \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} x_{ij}\text{ } p^{(j+1)}$$

In [ ]:
import numpy as np
import pandas as pd
from pulp import *

In [39]:
P = {j: p**j for j in range(1, n_tags+2)}  # p^j coefficients
M = 1000     # A sufficiently large number for the constraints


In [40]:

def max_expected_A(n_tags, m_msgs, p):
   
    # Create a PuLP problem with maximization
    prob = LpProblem("Maximize_Expected_A", LpMaximize)

    # Define the binary variables k indexed by (i, j)
    k = {(i, j): LpVariable(f'k_{i}_{j}', cat='Binary') for i in range(1, m_msgs+1) for j in range(1, n_tags+1)}

    # Define the binary variables w indexed by j
    w = {j: LpVariable(f'w_{j}', cat='Binary') for j in range(1, n_tags+1)}

    # Define the continuous variables z indexed by j
    z = {j: LpVariable(f'z_{j}', cat='Integer', lowBound=0) for j in range(1, n_tags+1)}

    # Define the binary variables x indexed by (i, j)
    x = {(i, j): LpVariable(f'x_{i}_{j}', cat='Binary') for i in range(1, m_msgs+1) for j in range(1, n_tags+1)}

    # Constraints
    for j in range(1, n_tags+1):
        prob += lpSum(k[i, j] for i in range(1, m_msgs+1)) >= 1, f"Tag_{j}_constraint"

    for i in range(1, m_msgs+1):
        prob += lpSum(k[i, j] for j in range(1, n_tags+1)) >= 1, f"Message_{i}_constraint"

    for j in range(1, n_tags+1):
        prob += z[j] == lpSum(k[i, j] for i in range(1, m_msgs+1)), f"Z_{j}_definition"
        prob += z[j] >= 1, f"Z_{j}_non_zero"
        
    prob += lpSum(w[j] for j in range(1, n_tags+1)) == 1, "Sum_w_equals_1"

    for j in range(1, n_tags+1):
        prob += z[j] - j <= M * w[j], f"BigM_1_{j}"
        prob += j - z[j] <= M * (1 - w[j]), f"BigM_2_{j}"

    for i in range(1, m_msgs+1):
        for j in range(1, n_tags+1):
            prob += x[i, j] <= k[i, j], f"x_{i}_{j}_leq_k_{i}_{j}"
            prob += x[i, j] <= w[j], f"x_{i}_{j}_leq_w_{j}"
            prob += x[i, j] >= k[i, j] + w[j] - 1, f"x_{i}_{j}_geq_k_{i}_{j}_plus_w_{j}_minus_1"


    # Objective Function
    prob += lpSum(x[i, j] * P[j+1] for i in range(1, m_msgs+1) for j in range(1, n_tags+1)), "Maximize_E_A"

    # Solve the problem
    prob.solve()

    # Extract the optimal values of k_{ij}
    k_values = np.zeros((m_msgs, n_tags), dtype=int)
    for i in range(1, m_msgs+1):
        for j in range(1, n_tags+1):
            k_values[i-1, j-1] = value(k[i, j])

    k_df = pd.DataFrame(k_values, columns=[f'k_{j}' for j in range(1, n_tags+1)], index=[f'Msg_{i}' for i in range(1, m_msgs+1)])
    k_df.columns = [f't{i}' for i in range(1,n_tags+1)]
    k_df.index = [f'm{i}' for i in range(1,m_msgs+1)]
    print(k_df)

    # Print the variable values
    for v in prob.variables():
        print(f"{v.name} = {v.varValue}")

    # Print the objective value
    print(f"Objective value: {value(prob.objective)}")
    
    return k_df


In [ ]:
n_tags = 10  
m_msgs = 15   
p = 0.9     
max_expected_A(n_tags, m_msgs, p)

In [ ]:
P = {j: p**j for j in range(1, n_tags+2)}  # Example values, replace with actual p values
P